In [1]:
import numpy as np
import pandas as pd

# Read IMDB Datasets
- Rating dataset
- Title dataset

In [2]:
netflix_data = pd.read_csv("netflix_titles.csv")
imdb_data = pd.read_csv("title_basic_imdb.tsv", sep = '\t')

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3169: DtypeWarning: Columns (4,5) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
imdb_data

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0000001,short,Carmencita,Carmencita,0,1894,\N,1,"Documentary,Short"
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,\N,5,"Animation,Short"
2,tt0000003,short,Pauvre Pierrot,Pauvre Pierrot,0,1892,\N,4,"Animation,Comedy,Romance"
3,tt0000004,short,Un bon bock,Un bon bock,0,1892,\N,12,"Animation,Short"
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893,\N,1,"Comedy,Short"
...,...,...,...,...,...,...,...,...,...
8841185,tt9916848,tvEpisode,Episode #3.17,Episode #3.17,0,2010,\N,\N,"Action,Drama,Family"
8841186,tt9916850,tvEpisode,Episode #3.19,Episode #3.19,0,2010,\N,\N,"Action,Drama,Family"
8841187,tt9916852,tvEpisode,Episode #3.20,Episode #3.20,0,2010,\N,\N,"Action,Drama,Family"
8841188,tt9916856,short,The Wind,The Wind,0,2015,\N,27,Short


In [4]:
imdb_rating = pd.read_csv("movie_ratings.tsv", sep = "\t")

In [5]:
imdb_rating

,tconst,averageRating,numVotes
0,tt0000001,5.7,1871
1,tt0000002,5.9,247
2,tt0000003,6.5,1646
3,tt0000004,5.8,159
4,tt0000005,6.2,2472
...,...,...,...
1233152,tt9916690,6.5,6
1233153,tt9916720,5.7,202
1233154,tt9916730,8.7,6
1233155,tt9916766,6.7,19


# Merge IMDB Datasets
Since 'imdb_data' does not have ratings, assign ratings from 'imdb_rating'

In [ ]:
new_dataframe_with_rating = pd.merge(imdb_data.set_index('tconst'), imdb_rating.set_index('tconst'), left_index=True, right_index=True).drop_duplicates()

In [ ]:
new_dataframe_with_rating.head(5)

In [ ]:
#Exclude TV Episodes
new_dataframe_with_rating = new_dataframe_with_rating[new_dataframe_with_rating['titleType']!='tvEpisode']

In [ ]:
netflix_data['title'] = netflix_data['title'].str.lower()
new_dataframe_with_rating['primaryTitle'] = new_dataframe_with_rating['primaryTitle'].str.lower()
new_dataframe_with_rating = new_dataframe_with_rating.rename(columns={"primaryTitle":"title"})

In [ ]:
#Reference: https://stackoverflow.com/questions/33961028/remove-non-numeric-rows-in-one-column-with-pandas
new_dataframe_with_rating = new_dataframe_with_rating[new_dataframe_with_rating.startYear.apply(lambda x: str(x).isnumeric())]
new_dataframe_with_rating['startYear'] = new_dataframe_with_rating['startYear'].astype(int)
new_dataframe_with_rating = new_dataframe_with_rating.rename(columns={"startYear":"year"})

# Merge Netflix titles and Rated IMDB Titles

In [ ]:
new_data_netflix = pd.merge(netflix_data,new_dataframe_with_rating, left_on = ['title','release_year'], right_on = ['title', 'year'])


In [ ]:
new_data_netflix.head(2)

In [ ]:
reduced_data = new_data_netflix.drop(columns=['titleType', 'date_added', 'description','runtimeMinutes','isAdult','endYear'])
reduced_data.head()

In [ ]:
reduced_data = reduced_data.drop(columns=['country', 'listed_in', 'release_year'])


In [ ]:
reduced_data.to_csv('netflix_ratings_data.csv', sep=',', index=False)
